In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, to_date, to_timestamp, hour, floor, datediff, current_date, min, lower, trim, count, udf, date_format
from pyspark.sql.types import StringType, DateType, TimestampType, IntegerType
import subprocess
import sys

# Função para instalar pacotes
def install_package(package_with_version):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_with_version])
        print(f"'{package_with_version}' instalado com sucesso.")
    except Exception as e:
        print(f"Erro ao instalar '{package_with_version}': {e}")
        sys.exit(1)


# Instalar as bibliotecas necessárias com versões compatíveis com Python 3.7
# 'Literal' foi introduzido no Python 3.8. Versões mais recentes de gspread/google-auth-oauthlib
# podem depender dele. Estamos fixando versões mais antigas.
install_package("pandas")
install_package("gspread==3.6.0") # Versão compatível com Python 3.7
install_package("google-auth-oauthlib==0.4.6") # Versão compatível com Python 3.7
install_package("boto3")
# --- Fim da seção de instalação de bibliotecas ---

# --- Importação de bibliotecas ---
import os
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import boto3
import datetime # Importado para verificar o tipo datetime.date
        

# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Lista de arquivos CSV a serem lidos
csv_files = [
    'dados_2025_06_02.csv',
    'dados_2025_05_27.csv',
    'dados_2025_05_28.csv',
    'dados_2025_05_29.csv',
    'dados_2025_05_30.csv',
    'dados_2025_05_31.csv',
    'dados_2025_06_01.csv',
    'dados_2025_05_20.csv',
    'dados_2025_05_21.csv',
    'dados_2025_05_22.csv',
    'dados_2025_05_23.csv',
    'dados_2025_05_24.csv',
    'dados_2025_05_25.csv',
    'dados_2025_05_26.csv'
]

# Prefixo do caminho S3
s3_prefix = 's3a://bucket-raw-upa-connect-mateus/'

# Criar a lista completa de caminhos S3
s3_paths = [s3_prefix + file for file in csv_files]

# Ler os CSVs necessários a partir da lista de caminhos
TabelaCompleta = spark.read.option('delimiter', ',') \
                             .option('header', 'true') \
                             .option('nullValue', 'null') \
                             .csv(s3_paths) # Agora lê múltiplos arquivos

paciente_df = spark.read.option('delimiter', ',') \
                         .option('header', 'true') \
                         .csv('s3a://bucket-trusted-upa-connect-mateus/paciente.csv')

print("--- DataFrames Iniciais ---")
print("Schema de TabelaCompleta:")
TabelaCompleta.printSchema()
print("Amostra de TabelaCompleta:")
TabelaCompleta.show(5, truncate=False)

print("\nSchema de paciente_df:")
paciente_df.printSchema()
print("Amostra de paciente_df:")
paciente_df.show(5, truncate=False)

### **Processamento para o Sensor 5 (Biometria)**

print("\n--- Processamento para o Sensor 5 (Biometria) ---")

# 1. Filtrar pelo sensor 5 e remover valores nulos em 'valor'
df_biometria = TabelaCompleta.filter(col("fk_sensor") == 5) \
                             .filter(col('valor').isNotNull())

print("\nDataFrame df_biometria após filtro e remoção de nulos:")
df_biometria.show(5, truncate=False)

# 2. Converter 'valor' para StringType (biometria), 'fk_upa' para IntegerType e 'data_hora' para Timestamp
# Normalizar a coluna 'valor' (biometria) para join
df_biometria = df_biometria.withColumn('valor', lower(trim(col('valor'))).cast(StringType())) \
                           .withColumn('fk_upa', col('fk_upa').cast(IntegerType())) \
                           .withColumn('data_hora', to_timestamp(col('data_hora'))) # Converter para Timestamp

print("\nDataFrame df_biometria após cast e normalização de 'valor':")
df_biometria.show(5, truncate=False)
df_biometria.printSchema()


# 3. Agrupar por biometria e dia, pegando a primeira ocorrência do dia e o fk_upa
df_biometria_agrupado = df_biometria.groupBy(
    'valor',
    to_date(col('data_hora')).alias('data_biometria') # Renomeado para data_biometria
).agg(
    min(col('data_hora')).alias('primeira_data_hora_dia'),
    min(col('fk_upa')).alias('fk_upa') # Manter o fk_upa do registro original
).orderBy('data_biometria', 'valor') # Ordenar por data_biometria

print("\nDataFrame df_biometria_agrupado:")
df_biometria_agrupado.show(5, truncate=False)
df_biometria_agrupado.printSchema()

# REMOVIDA: UDF para decodificar a biometria, pois o CSV já está arrumado.

# 4. Juntar com paciente_df APENAS para obter a data_nascimento
# Normalizar a coluna 'biometria' em paciente_df para join (agora sem decodificação)
paciente_df_normalized = paciente_df.withColumn('biometria_join', lower(trim(col('biometria'))).cast(StringType()))

# Adicionado para visualizar a biometria normalizada no paciente_df_normalized
print("\nDataFrame paciente_df_normalized com 'biometria_join' (após normalização):")
# Removido 'biometria_cleaned' do select, pois a UDF foi removida
paciente_df_normalized.select('biometria', 'biometria_join', 'nome', 'data_nascimento').show(10, truncate=False)


df_biometria_final = df_biometria_agrupado.join(
    paciente_df_normalized.select('biometria_join', 'data_nascimento', 'nome', 'cpf'), # Selecionar 'nome'
    df_biometria_agrupado.valor == paciente_df_normalized.biometria_join,
    "left_outer" # Usar left_outer para manter todas as biometrias, mesmo que não encontre paciente
).withColumnRenamed("nome", "nome_paciente") \
 .withColumnRenamed("cpf", "cpf_paciente") \
 .drop("biometria_join") # A coluna biometria_join não é mais necessária após o join

print("\nDataFrame df_biometria_final após o JOIN (antes de calcular idade):")
df_biometria_final.show(10, truncate=False) # Mostrar mais linhas para ver os nulos
df_biometria_final.printSchema()

# Verificar contagem de nulos após o join
print("\nContagem de nulos em colunas chave após o JOIN:")
df_biometria_final.select(
    count(when(col("nome_paciente").isNull(), 1)).alias("nulos_nome_paciente"),
    count(when(col("data_nascimento").isNull(), 1)).alias("nulos_data_nascimento"),
    count(when(col("cpf_paciente").isNull(), 1)).alias("nulos_cpf_paciente")
).show()

# 5. Calcular Idade e Faixa Etária
# Assegurar que 'data_nascimento' é tipo Data.
# Tentaremos múltiplos formatos de data. Se nenhum funcionar, resultará em NULL.
df_biometria_final = df_biometria_final.withColumn(
    "data_nascimento_dt",
    when(to_date(col("data_nascimento"), "yyyy-MM-dd").isNotNull(), to_date(col("data_nascimento"), "yyyy-MM-dd"))
    .when(to_date(col("data_nascimento"), "dd/MM/yyyy").isNotNull(), to_date(col("data_nascimento"), "dd/MM/yyyy"))
    .when(to_date(col("data_nascimento"), "MM/dd/yyyy").isNotNull(), to_date(col("data_nascimento"), "MM/dd/yyyy"))
    .otherwise(lit(None).cast(DateType())) # Caso nenhum formato corresponda
)
df_biometria_final = df_biometria_final.withColumn(
    "IdadeCorrigida", floor(datediff(current_date(), col("data_nascimento_dt")) / 365.25)
)
# Renomear IdadeCorrigida para idade_paciente
df_biometria_final = df_biometria_final.withColumnRenamed("IdadeCorrigida", "idade_paciente")


df_biometria_final = df_biometria_final.withColumn("Faixa_Etaria",
    when((col("idade_paciente") >= 0) & (col("idade_paciente") <= 12), "0–12") # Usar idade_paciente
    .when((col("idade_paciente") >= 13) & (col("idade_paciente") <= 17), "13–17") # Usar idade_paciente
    .when((col("idade_paciente") >= 18) & (col("idade_paciente") <= 39), "18–39") # Usar idade_paciente
    .when((col("idade_paciente") >= 40) & (col("idade_paciente") <= 59), "40–59") # Usar idade_paciente
    .otherwise("60+"))

# 6. Criar Faixa de Horários
df_biometria_final = df_biometria_final.withColumn("Hora", hour(col("primeira_data_hora_dia")))
df_biometria_final = df_biometria_final.withColumn("Faixa_Hora",
    when((col("Hora") >= 0) & (col("Hora") < 6), "0–6h")
    .when((col("Hora") >= 6) & (col("Hora") < 9), "6–9h")
    .when((col("Hora") >= 9) & (col("Hora") < 12), "9–12h")
    .when((col("Hora") >= 12) & (col("Hora") < 15), "12–15h")
    .when((col("Hora") >= 15) & (col("Hora") < 18), "15–18h")
    .when((col("Hora") >= 18) & (col("Hora") < 21), "18–21h")
    .otherwise("21–24h"))

# 7. Selecionar e ordenar as colunas finais
colunas_biometria_ordenadas = [
    'valor',
    'nome_paciente',
    'idade_paciente', # RENOMEADA
    'Faixa_Etaria',
    'data_biometria', # RENOMEADA E REORDENADA
    date_format(col('primeira_data_hora_dia'), 'HH:mm:ss').alias('hora_biometria'), # Apenas a hora, minuto e segundo, e RENOMEADA
    'Faixa_Hora',
    'fk_upa'
]

df_biometria_final = df_biometria_final.select(colunas_biometria_ordenadas)

print("\n--- DataFrame Final de Biometria ---")
df_biometria_final.show(120, truncate=False)

# Salvar o DataFrame de biometria em CSV no S3
output_path_biometria = 's3a://bucket-trusted-upa-connect-mateus/biometria_paciente.csv'
print(f"\nSalvando DataFrame de biometria para: {output_path_biometria}")
df_biometria_final.coalesce(1) \
                  .write \
                  .option('header', 'true') \
                  .mode('overwrite') \
                  .csv(output_path_biometria)
print("DataFrame de biometria salvo com sucesso no S3.")

### **Enviar DataFrame para Google Sheets**

# === CONFIGURAÇÕES PARA O GOOGLE SHEETS ===
GOOGLE_SHEET_ID = '1i6BfuZXPOcTp6BFAiVkpktOBym0HtakZacUKfDzu1zI' # Use o ID da sua planilha
S3_BUCKET_CREDENTIALS = 'bucket-trusted-upa-connect-mateus'
S3_KEY_CREDENTIALS = 'credenciais.json'
LOCAL_CREDENTIALS_PATH = '/tmp/credenciais.json'

# === AUTENTICAÇÃO GOOGLE ===
print(f"\nBaixando credenciais do S3: s3://{S3_BUCKET_CREDENTIALS}/{S3_KEY_CREDENTIALS} para {LOCAL_CREDENTIALS_PATH}...")
s3 = boto3.client('s3')
try:
    s3.download_file(S3_BUCKET_CREDENTIALS, S3_KEY_CREDENTIALS, LOCAL_CREDENTIALS_PATH)
    print("Credenciais baixadas com sucesso.")
except Exception as e:
    print(f"Erro ao baixar credenciais do S3: {e}")
    sys.exit(1)

scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
credenciais = Credentials.from_service_account_file(LOCAL_CREDENTIALS_PATH, scopes=scopes)
cliente = gspread.authorize(credenciais)
planilha = cliente.open_by_key(GOOGLE_SHEET_ID)

# === ENVIA O DATAFRAME DE BIOMETRIA PARA A NOVA ABA "BiometriaPaciente" ===
NOME_ABA_BIOMETRIA_PACIENTE = "BiometriaPaciente"
print(f"\nEnviando DataFrame de biometria para a aba '{NOME_ABA_BIOMETRIA_PACIENTE}'...")

# Converter Spark DataFrame df_biometria_final para Pandas DataFrame
try:
    df_biometria_pandas = df_biometria_final.toPandas()
    print("DataFrame Spark de biometria convertido para Pandas com sucesso.")
except Exception as e:
    print(f"Erro ao converter Spark DataFrame de biometria para Pandas: {e}")
    sys.exit(1)

# Ajustar tipagem das colunas para o Google Sheets
# Coluna 'data_biometria': assegurar que é string DD/MM/YYYY
if 'data_biometria' in df_biometria_pandas.columns:
    # Verifica se a coluna ainda é do tipo datetime.date (ou similar) e a formata.
    # Caso contrário, se já for string, assume que está ok e apenas preenche nulos.
    if pd.api.types.is_object_dtype(df_biometria_pandas['data_biometria']):
        df_biometria_pandas['data_biometria'] = df_biometria_pandas['data_biometria'].apply(
            lambda x: x.strftime('%d/%m/%Y') if isinstance(x, (datetime.date, datetime.datetime)) else x
        )
    df_biometria_pandas['data_biometria'] = df_biometria_pandas['data_biometria'].fillna('')


# Coluna 'hora_biometria': (já vem como string 'HH:mm:ss' do Spark)
if 'hora_biometria' in df_biometria_pandas.columns:
    df_biometria_pandas['hora_biometria'] = df_biometria_pandas['hora_biometria'].fillna('')

# Coluna 'data_nascimento' não está mais no df_biometria_final, então a lógica para ela foi removida.
# A coluna 'IdadeCorrigida' foi renomeada para 'idade_paciente' e já é numérica, não necessita de tratamento especial aqui.

dados_biometria = [df_biometria_pandas.columns.tolist()] + df_biometria_pandas.values.tolist()

try:
    aba_biometria = planilha.worksheet(NOME_ABA_BIOMETRIA_PACIENTE)
    aba_biometria.clear()
    print(f"Aba '{NOME_ABA_BIOMETRIA_PACIENTE}' encontrada e limpa.")
except gspread.exceptions.WorksheetNotFound:
    aba_biometria = planilha.add_worksheet(title=NOME_ABA_BIOMETRIA_PACIENTE, rows=str(len(dados_biometria) + 100), cols=str(len(df_biometria_pandas.columns) + 10))
    print(f"Aba '{NOME_ABA_BIOMETRIA_PACIENTE}' criada.")

try:
    aba_biometria.update(dados_biometria)
    print(f"Dados do DataFrame de biometria enviados com sucesso para a aba '{NOME_ABA_BIOMETRIA_PACIENTE}'.")
except Exception as e:
    print(f"Erro ao atualizar a aba do Google Sheets com dados de biometria: {e}")

# Opcional: Remover o arquivo de credenciais temporário
if os.path.exists(LOCAL_CREDENTIALS_PATH):
    os.remove(LOCAL_CREDENTIALS_PATH)
    print(f"Arquivo de credenciais temporário '{LOCAL_CREDENTIALS_PATH}' removido.")

# Parar a sessão Spark
spark.stop()
print("\nSessão Spark parada.")


'pandas' instalado com sucesso.


'gspread==3.6.0' instalado com sucesso.


'google-auth-oauthlib==0.4.6' instalado com sucesso.


'boto3' instalado com sucesso.
:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-baa17a85-d3cc-4531-9cb5-5cd9ed114d45;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 313ms :: artifacts dl 19ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

...

25/06/13 20:08:08 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/06/13 20:08:09 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


DataFrame de biometria salvo com sucesso no S3.

Baixando credenciais do S3: s3://bucket-trusted-upa-connect-mateus/credenciais.json para /tmp/credenciais.json...


/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Credenciais baixadas com sucesso.

Enviando DataFrame de biometria para a aba 'BiometriaPaciente'...


DataFrame Spark de biometria convertido para Pandas com sucesso.
Aba 'BiometriaPaciente' encontrada e limpa.
Dados do DataFrame de biometria enviados com sucesso para a aba 'BiometriaPaciente'.
Arquivo de credenciais temporário '/tmp/credenciais.json' removido.

Sessão Spark parada.
